# Camera Trap Processing Thesis

Author: Jameson Andrews\
EDITS:\
09/13/2023, Initial Creation, Jameson

## Load model

In [193]:
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras import models
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.applications.vgg19 import preprocess_input
from keras.applications.vgg19 import decode_predictions
from keras.applications.vgg19 import VGG19

preTrainedModel = VGG19()
preTrainedModel.trainable = False
preTrainedModel.summary()


Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_27 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

2023-09-28 19:21:42.657745: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 671ms/step
[[('n02395406', 'hog', 0.14550796), ('n02797295', 'barrow', 0.040324006), ('n03393912', 'freight_car', 0.024490995), ('n02437616', 'llama', 0.02321001), ('n02793495', 'barn', 0.021253262)]]


## Check basic VGG19 Classification

In [218]:
testPicture = load_img('../noDeerDay.JPG', target_size=(224,224))
# Convert Image pixels to a numpy array to be operated on
pictureArray = img_to_array(testPicture)
# reshape Data for model
pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
# prepare the image for the VGG model
pictureArray = preprocess_input(pictureArray)
# predict the probability across all output classes
yhat = preTrainedModel.predict(pictureArray)
# convert the probabilities to class labels
labels = decode_predictions(yhat)
# retrieve the most likely result, e.g. highest probability
label = labels[0][0]
# print the classification
print(labels)



1/1 [==============================] - 0s 97ms/step
[[('n04371774', 'swing', 0.16665962), ('n03967562', 'plow', 0.06998367), ('n04208210', 'shovel', 0.067548245), ('n04604644', 'worm_fence', 0.06564355), ('n03141823', 'crutch', 0.04753662)]]


## Create complete Model with Custom output Layer

In [195]:
from keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam

completeModel = models.Sequential()
completeModel.add(preTrainedModel)
completeModel.add(Dense(100, activation = "relu"))
completeModel.add(Dense(2, activation = "softmax"))
completeModel.summary()
completeModel.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])


Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 1000)              143667240 
                                                                 
 dense_47 (Dense)            (None, 100)               100100    
                                                                 
 dense_48 (Dense)            (None, 2)                 202       
                                                                 
Total params: 143,767,542
Trainable params: 100,302
Non-trainable params: 143,667,240
_________________________________________________________________


## Orgainze test and training data

In [196]:
trainImages = keras.utils.image_dataset_from_directory( #Takes given directory and automatically organizes it based on subdirectories
    directory='trainingData',
    labels="inferred",
    label_mode='categorical',
    batch_size=32,
    image_size=(224, 224))
testImages = keras.utils.image_dataset_from_directory(
    directory='testData',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(224, 224))

2841
Found 3509 files belonging to 2 classes.
Found 305 files belonging to 2 classes.


## Prediction before training

In [197]:
testPicture = load_img('../deerdownload.jpg', target_size=(224,224))
pictureArray = img_to_array(testPicture)
pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
pictureArray = preprocess_input(pictureArray)
yhat = completeModel.predict(pictureArray)
print(yhat)

2023-09-28 19:21:43.992866: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 625ms/step
[[0.49757993 0.50242007]]


## Train Last two layers

In [208]:
completeModel.fit(trainImages, epochs=20, batch_size=100)

Epoch 1/20


110/110 [==============================] - 39s 351ms/step - loss: 0.1086 - accuracy: 0.9575
Epoch 2/20
110/110 [==============================] - 39s 352ms/step - loss: 0.1044 - accuracy: 0.9598
Epoch 3/20
110/110 [==============================] - 39s 351ms/step - loss: 0.1003 - accuracy: 0.9621
Epoch 4/20
110/110 [==============================] - 39s 355ms/step - loss: 0.0957 - accuracy: 0.9644
Epoch 5/20
110/110 [==============================] - 39s 350ms/step - loss: 0.0933 - accuracy: 0.9649
Epoch 6/20
110/110 [==============================] - 39s 349ms/step - loss: 0.0901 - accuracy: 0.9655
Epoch 7/20
110/110 [==============================] - 39s 349ms/step - loss: 0.0862 - accuracy: 0.9655
Epoch 8/20
110/110 [==============================] - 39s 350ms/step - loss: 0.0837 - accuracy: 0.9667
Epoch 9/20
110/110 [==============================] - 39s 350ms/step - loss: 0.0820 - accuracy: 0.9701
Epoch 10/20
110/110 [==============================] - 39s 349ms/step - loss: 0.0786

## Test Some data

In [209]:
score = completeModel.evaluate(testImages)
print ("loss = ", score[0])
print ("accuracy = ", score[1])

10/10 [==============================] - 4s 338ms/step - loss: 0.5413 - accuracy: 0.8820
loss =  0.5412991642951965
accuracy =  0.8819671869277954


## See what happens with a deer image

In [217]:
testPicture = load_img('../deerdownload.jpg', target_size=(224,224))
pictureArray = img_to_array(testPicture)
pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
pictureArray = preprocess_input(pictureArray)
yhat = completeModel.predict(pictureArray)
print(yhat)


# 0 is deer 1 is no deer, i need to figure out how to change this

1/1 [==============================] - 0s 16ms/step
[[0.98900723 0.01099278]]


## Save the model for export

In [ ]:
completeModel.save("alphaModel.keras")